In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = '16'
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# --- Código para cargar y pivotar los datos ---
db_file_path = 'planta_quimica.db'
conn = sqlite3.connect(db_file_path)
query = """
    SELECT
        timestamp,
        MAX(CASE WHEN sensor_id = 'temp_1' THEN valor END) AS temp_1,
        MAX(CASE WHEN sensor_id = 'temp_2' THEN valor END) AS temp_2,
        MAX(CASE WHEN sensor_id = 'temp_3' THEN valor END) AS temp_3,
        MAX(CASE WHEN sensor_id = 'pres_1' THEN valor END) AS pres_1,
        MAX(CASE WHEN sensor_id = 'pres_2' THEN valor END) AS pres_2
    FROM
        lecturas_sensores
    GROUP BY
        timestamp;
"""
df_ancho = pd.read_sql_query(query, conn)
conn.close()
# ------------------------------------------------

# 1. Preparar los datos para el clustering
#    Nos quedamos solo con las columnas numéricas y las estandarizamos
features = df_ancho.drop('timestamp', axis=1)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)


# 2. Calcular Codo y Silueta para diferentes valores de K
k_values = range(2, 11) # Probaremos de 2 a 10 clusters
inertia_values = []
silhouette_values = []

print("Calculando scores para diferentes números de clusters...")
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(features_scaled)
    
    # Para el método del codo
    inertia_values.append(kmeans.inertia_)
    
    # Para el análisis de silueta
    silhouette_values.append(silhouette_score(features_scaled, kmeans.labels_))

# 3. Graficar los resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico del Codo
ax1.plot(k_values, inertia_values, 'bo-')
ax1.set_title('Método del Codo')
ax1.set_xlabel('Número de Clusters (K)')
ax1.set_ylabel('Inercia')

# Gráfico de Silueta
ax2.plot(k_values, silhouette_values, 'ro-')
ax2.set_title('Análisis de Silueta')
ax2.set_xlabel('Número de Clusters (K)')
ax2.set_ylabel('Score de Silueta')

plt.show()

# 1. Crear y entrenar el modelo K-Means final con K=6
print("Entrenando el modelo K-Means con 6 clusters...")
kmeans_final = KMeans(n_clusters=6, random_state=42, n_init=10)
kmeans_final.fit(features_scaled)

# 2. Añadir las etiquetas del cluster a nuestra tabla original
df_ancho['cluster'] = kmeans_final.labels_

# 3. Mostrar las primeras filas con la nueva columna 'cluster'
print("\nTabla con la columna de cluster añadida:")
print(df_ancho.head())

# 4. Contar cuántos puntos hay en cada cluster
print("\nDistribución de puntos por cluster:")
print(df_ancho['cluster'].value_counts())

# 1. Seleccionar solo las columnas numéricas antes de agrupar
df_numerico = df_ancho.drop('timestamp', axis=1)

# 2. Agrupar por la columna 'cluster' y calcular el promedio
cluster_profiles = df_numerico.groupby('cluster').mean()

# 3. Imprimir la tabla de perfiles
print("--- Perfiles de los Clusters (Valores Promedio de Sensores) ---")
print(cluster_profiles)

# 1. Entrenar el modelo final con K=6
kmeans_final = KMeans(n_clusters=6, random_state=42, n_init=10)
kmeans_final.fit(features_scaled)

# 2. Calcular la distancia de cada punto al centroide de su cluster
#    Esto nos da una "puntuación de anomalía"
distancias = kmeans_final.transform(features_scaled)
distancia_minima = np.min(distancias, axis=1)
df_ancho['distancia_anomalia'] = distancia_minima

# 3. Ordenar el DataFrame para encontrar las lecturas más anómalas
df_anomalias = df_ancho.sort_values(by='distancia_anomalia', ascending=False)

# 4. Mostrar las 5 lecturas más anómalas
print("--- TOP 5 LECTURAS MÁS ANÓMALAS DETECTADAS ---")
print(df_anomalias.head(5))